In [ ]:
%run "/Users/rayannitcheu/Documents/GitHub/OilDesk-Intern-Assessment/solutions/exercice_3.ipynb"

In [ ]:
import asyncio
import aiosqlite
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

In [ ]:
async def async_add_non_existing_columns():
    async with aiosqlite.connect('metal_prices.db') as db:
        cursor = await db.cursor()

        # Vérifier si la colonne 'rsi' existe
        res = await cursor.execute("PRAGMA table_info(metal_prices)")
        existing_columns = [c[1] for c in await res.fetchall()] # Extracting the name of the column from the resulting tuples 

        if 'rsi' not in existing_columns:
            await cursor.execute("ALTER TABLE metal_prices ADD COLUMN rsi FLOAT")

        if 'macd' not in existing_columns:
            await cursor.execute("ALTER TABLE metal_prices ADD COLUMN macd FLOAT")

        await db.commit()

In [1]:
# Créer un moteur SQLAlchemy asynchrone
async_engine = create_async_engine('sqlite+aiosqlite:///metal_prices.db')
SessionAsync = sessionmaker(async_engine, class_=AsyncSession, expire_on_commit=False)

SyntaxError: invalid syntax (2638770353.py, line 2)

In [ ]:
async def async_update_indicators(data, indicator):
    async with SessionAsync() as session:
        if isinstance(data, list):
            for d in data:
                res = await session.execute(select(MetalTable).filter_by(date=d.get("Date"), metal=d.get("Metal")))
                record = res.all()
                if record:
                    if "RSI" in indicator:
                        record[0][0].rsi = d.get(indicator)
                    if "MACD" in indicator:
                        record[0][0].macd = d.get(indicator)
        else:
            res = await session.execute(
                select(MetalTable).filter_by(date=data.get("Date"), metal=data.get("Metal")))
            record = res.all()
            if record:
                if indicator == "RSI":
                    record[0][0].rsi = data.get(indicator)
                if indicator == "MACD":
                    record[0][0].macd = data.get(indicator)

        await session.commit()


async def async_update_commo(commo):
    # Ces fonctions sont supposées être définies ailleurs dans votre code
    df_rsi = data_w_indicators[[f"{commo}-RSI"]].iloc[rsi_window:, :]
    df_rsi.columns = [commo]

    df_macd = data_w_indicators[[f"{commo}-MACD"]].iloc[max_window:, :]
    df_macd.columns = [commo]

    sql_rsi = convert_df_to_sql(df_rsi, "RSI")
    sql_macd = convert_df_to_sql(df_macd, "MACD")

    await async_update_indicators(sql_rsi, "RSI")
    await async_update_indicators(sql_macd, "MACD")


async def async_read_from_db():
    async with SessionAsync() as session:
        res = await session.execute(select(MetalTable))
        return res.scalars().all()


async def main():
    # Ajouter les colonnes manquantes
    await async_add_non_existing_columns()

    # Traiter toutes les matières premières de manière asynchrone
    tasks = [async_update_commo(commo) for commo in commodities_list]
    await asyncio.gather(*tasks)

    # Lire depuis la base de données 5 fois de manière concurrente
    reading_tasks = [async_read_from_db() for _ in range(5)]
    reading_res = await asyncio.gather(*reading_tasks)

    # Afficher les résultats de lecture
    for i, res in enumerate(reading_res, 1):
        print(f"Lecture {i}: {res}")

In [ ]:
asyncio.run(main())